# Translate MAT files containing musing data to Python

The Music Synthesis lab requires MATAB mat-files to describe the sheet music. 

These MAT-files are located at https://dspfirst.gatech.edu/chapters/DSP1st2eLabs/bach_fugueData.zip

This notebook translates them to Python format.

In [2]:
import scipy.io as sio
import numpy as np

## Read in the MAT file

In [26]:
file_base = 'bach_fugue'

mat_file = file_base + '.mat'

mat_dict = sio.loadmat(mat_file)

### Let's see what's in there

The data we read from the MAT-file are in the `mat_dict` dict. Let's print that out. 

In [27]:
mat_dict

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Sun Sep 23 14:54:46 2001',
 '__version__': '1.0',
 '__globals__': [],
 'theVoices': array([[(array([[  3,   4,   5,   7,   9,  11,  12,  13,  15,  17,  19,  20,  21,
                  23,  25,  26,  27,  31,  32,  33,  34,  35,  36,  37,  38,  39,
                  40,  41,  43,  45,  47,  49,  51,  53,  55,  57,  59,  61,  63,
                  65,  70,  71,  72,  73,  74,  75,  78,  79,  80,  81,  82,  83,
                  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97, 107, 109,
                 111, 115, 117, 119, 121, 123, 124, 125, 127, 129, 135, 137, 139,
                 140, 141, 143, 145, 151, 153, 155, 156, 157, 159, 161, 167, 169,
                 171, 173, 175, 179, 181, 183, 187, 188, 189, 191, 193, 195, 197,
                 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223,
                 225, 227, 228, 229, 231, 233]], dtype=uint8), array([[1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2, 

The important data are stored under the key `theVoices`. Let's examine that more closely.

Note that `theVoices` has an extra dimension which we eliminate using `np.squeeze`.

In [28]:
voices = mat_dict['theVoices'].squeeze()

voices

array([(array([[  3,   4,   5,   7,   9,  11,  12,  13,  15,  17,  19,  20,  21,
                23,  25,  26,  27,  31,  32,  33,  34,  35,  36,  37,  38,  39,
                40,  41,  43,  45,  47,  49,  51,  53,  55,  57,  59,  61,  63,
                65,  70,  71,  72,  73,  74,  75,  78,  79,  80,  81,  82,  83,
                86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97, 107, 109,
               111, 115, 117, 119, 121, 123, 124, 125, 127, 129, 135, 137, 139,
               140, 141, 143, 145, 151, 153, 155, 156, 157, 159, 161, 167, 169,
               171, 173, 175, 179, 181, 183, 187, 188, 189, 191, 193, 195, 197,
               199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223,
               225, 227, 228, 229, 231, 233]], dtype=uint8), array([[1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2,
               1.2, 1. , 1. , 4. , 1. , 1. , 0.8, 1. , 1. , 1. , 1. , 1. , 1. ,
               1. , 1.2, 2. , 1.2, 1.2, 1.2, 2. , 1.2, 1.2, 2. , 

In [40]:
for k,v in voices.dtype.fields.items():
    print(k, v)

startPulses (dtype('O'), 0)
durations (dtype('O'), 8)
noteNumbers (dtype('O'), 16)


So, `voices` consists of three (one for each voice) arrays and each of these array holds three vectors; these three vectors are of equal length:
* one vector holds *normalized* tone durations measured in pulse
* a second holds the start time for the tone; again measured in pulses
* the last one holds the *tone number*, i.e., the number of the piano key that produces this tone

The order of these vectors can be determined from the information on the `dtype` field. We can determine this order as follows:

In [44]:
field_order = []

for k in voices.dtype.fields.keys():
    field_order.append(k)

field_order

['startPulses', 'durations', 'noteNumbers']

Now, for example, for the first voice we have:

In [45]:
for n in range(3):
    print(field_order[n], voices[0][n])


startPulses [[  3   4   5   7   9  11  12  13  15  17  19  20  21  23  25  26  27  31
   32  33  34  35  36  37  38  39  40  41  43  45  47  49  51  53  55  57
   59  61  63  65  70  71  72  73  74  75  78  79  80  81  82  83  86  87
   88  89  90  91  92  93  95  96  97 107 109 111 115 117 119 121 123 124
  125 127 129 135 137 139 140 141 143 145 151 153 155 156 157 159 161 167
  169 171 173 175 179 181 183 187 188 189 191 193 195 197 199 201 203 205
  207 209 211 213 215 217 219 221 223 225 227 228 229 231 233]]
durations [[1.  1.  1.2 1.2 1.2 1.  1.  1.2 1.2 1.2 1.  1.  1.2 1.2 1.  1.  4.  1.
  1.  0.8 1.  1.  1.  1.  1.  1.  1.  1.2 2.  1.2 1.2 1.2 2.  1.2 1.2 2.
  2.  2.  2.  4.  1.  1.  1.  1.  1.  3.  1.  1.  1.  1.  1.  3.  1.  1.
  1.  1.  1.  1.  1.  1.2 1.  1.  4.  2.  1.2 1.2 2.  1.2 1.2 1.2 1.  1.
  1.2 1.2 4.  1.2 1.2 1.  1.  1.2 1.2 4.  1.2 1.2 1.  1.  1.2 1.2 4.  1.2
  1.2 2.  1.2 1.2 2.  1.2 1.2 1.  1.  1.2 2.  1.2 2.  1.2 1.2 1.2 2.  1.2
  1.2 1.2 2.  1.2 1.2 1.2 2.  

### Repacking the data

We will repackage the data describing the music as follows:

* A list `theVoices` provides the top-level container, it will hold three elements (one for each voice)
* Each element of that list is a dictionary with three key-value pairs:
  - under key `durations` the array holding the note durations is stored
  - under key `startPulses` the array holding the start times is stored
  - under key `noteNumbers` the piano key that plays this note is stored
  - note that these keys are identical to those we extracted from `dtype`
  

In [49]:
theVoices = []

for v in voices:
    d = {}
    for n  in range(3):
        d[field_order[n]] = v[n].squeeze()
    theVoices.append(d)

theVoices


[{'startPulses': array([  3,   4,   5,   7,   9,  11,  12,  13,  15,  17,  19,  20,  21,
          23,  25,  26,  27,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  41,  43,  45,  47,  49,  51,  53,  55,  57,  59,  61,  63,
          65,  70,  71,  72,  73,  74,  75,  78,  79,  80,  81,  82,  83,
          86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97, 107, 109,
         111, 115, 117, 119, 121, 123, 124, 125, 127, 129, 135, 137, 139,
         140, 141, 143, 145, 151, 153, 155, 156, 157, 159, 161, 167, 169,
         171, 173, 175, 179, 181, 183, 187, 188, 189, 191, 193, 195, 197,
         199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223,
         225, 227, 228, 229, 231, 233], dtype=uint8),
  'durations': array([1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2,
         1.2, 1. , 1. , 4. , 1. , 1. , 0.8, 1. , 1. , 1. , 1. , 1. , 1. ,
         1. , 1.2, 2. , 1.2, 1.2, 1.2, 2. , 1.2, 1.2, 2. , 2. , 2. , 2. ,
         4. , 1. , 1. , 1. , 1

## Store data

To be able to load these data into the lab, we need to store them in a file. The most plausible format to use is Python's `pickle` format.

In [50]:
## Store data
import pickle

pkl_file = file_base + '.pkl'

with open(pkl_file, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(theVoices, f, pickle.HIGHEST_PROTOCOL)

Let's make sure this works.

In [51]:
with open(pkl_file, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

data

[{'startPulses': array([  3,   4,   5,   7,   9,  11,  12,  13,  15,  17,  19,  20,  21,
          23,  25,  26,  27,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  41,  43,  45,  47,  49,  51,  53,  55,  57,  59,  61,  63,
          65,  70,  71,  72,  73,  74,  75,  78,  79,  80,  81,  82,  83,
          86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97, 107, 109,
         111, 115, 117, 119, 121, 123, 124, 125, 127, 129, 135, 137, 139,
         140, 141, 143, 145, 151, 153, 155, 156, 157, 159, 161, 167, 169,
         171, 173, 175, 179, 181, 183, 187, 188, 189, 191, 193, 195, 197,
         199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223,
         225, 227, 228, 229, 231, 233], dtype=uint8),
  'durations': array([1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2, 1.2, 1.2, 1. , 1. , 1.2,
         1.2, 1. , 1. , 4. , 1. , 1. , 0.8, 1. , 1. , 1. , 1. , 1. , 1. ,
         1. , 1.2, 2. , 1.2, 1.2, 1.2, 2. , 1.2, 1.2, 2. , 2. , 2. , 2. ,
         4. , 1. , 1. , 1. , 1